In [7]:
from bs4 import BeautifulSoup
import csv
import logging
import pandas as pd
import requests
import time
import nltk

In [8]:
raw = pd.read_csv("raw.csv")  
    
raw

,created_published,date,dates_of_publication,description,essay,essay_contributor,language,latlong,location,raw_lccn,subjects,title,url
0,"['Abbeville, S.C. : Charles H. Allen, 1847-186...",1847,1847-1869,"['Weekly Vol. 4, no. 1 (Mar. 3, 1847)-v. 25, n...","<p>\n\tFor nearly a century, the <em>Abbeville...","['University of South Carolina; Columbia, SC']",['english'],"[34.17895, -82.38025]","['united states', 'south carolina', 'abbeville...",sn 85026945,"['Abbeville (S.C.)--Newspapers', 'Abbeville Co...",The Abbeville banner.,https://www.loc.gov/item/sn85026945/
1,"['Abbeville, S.C. : Hugh Wilson']",1865,1865-1865,['Weekly Began in July 1865. Ceased with Aug. ...,<p>\n\tThe short-lived weekly <em>Abbeville Bu...,"['University of South Carolina; Columbia, SC']",['english'],"[34.17895, -82.38025]","['united states', 'abbeville county', 'abbevil...",sn 93067670,"['Abbeville County (S.C.)--Newspapers', 'South...",The Abbeville bulletin.,https://www.loc.gov/item/sn93067670/
2,"['Abbeville, S.C. : Bonham and Perrin']",1884,1884-1887,"['Weekly Began Oct. 1, 1884; ceased in 1887. C...",<p>\n\tThe <em>Abbeville Messenger</em> (1884-...,"['University of South Carolina; Columbia, SC']",['english'],"[34.17895, -82.38025]","['united states', 'abbeville county', 'abbevil...",sn 93067668,"['Abbeville County (S.C.)--Newspapers', 'South...",The Abbeville messenger.,https://www.loc.gov/item/sn93067668/
3,"['Abbeville, S.C. : W.A. Lee & Hugh Wilson, 18...",1869,1869-1924,"['Triweekly, Jan. 7, 1920-Feb. 13, 1924 Began ...","<p>\n\tFor nearly a century, the <em>Abbeville...","['University of South Carolina; Columbia, SC']",['english'],"[34.17895, -82.38025]","['united states', 'south carolina', 'abbeville...",sn 84026853,"['Abbeville (S.C.)--Newspapers', 'Abbeville Co...",The Abbeville press and banner.,https://www.loc.gov/item/sn84026853/
4,"['Abbeville, S.C. : W.A. Lee and Hugh Wilson, ...",1860,1860-1869,"['Weekly Vol. 8, no. 28 (Nov. 9, 1860)-v. 17, ...","<p>\n\tFor nearly a century, the <em>Abbeville...","['University of South Carolina; Columbia, SC']",['english'],"[34.17895, -82.38025]","['united states', 'south carolina', 'abbeville...",sn 85042527,"['Abbeville (S.C.)--Newspapers', 'Abbeville Co...",Abbeville press.,https://www.loc.gov/item/sn85042527/
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3675,"['Allegheny, Pa. : Narod. Hrv. Zajednica']",1894,1894-current,['Weekly (except the second week in July and t...,<p>\n\tThe&#160;<em>Zajedni</em><em>&#269;</em...,"['Ohio Historical Society, Columbus, OH']","['english', 'croatian']","[41.0981, -80.65079]","['mahoning', 'mahoning county', 'pennsylvania'...",sn 84024547,"['Croats--United States--Newspapers', 'Croatia...",Zajedničar = Fraternalist.,https://www.loc.gov/item/sn84024547/
3676,"['Chicago, Ill. : Polish National Alliance, 19...",1900,1900-1913,"['Began with rok 19, no. 1 (13 Stycznia, 1900)...","<p>\n\tIn 1881, <em>Zgoda</em> (&#8220;Harmony...",['University of Illinois at Urbana-Champaign L...,"['polish', 'english']","[41.88425, -87.63245]","['wisconsin', 'chicago', 'illinois', 'united s...",2017218620,"['Polish people--United States--Newspapers', '...",Zgoda : Wydania dla niewiast,https://www.loc.gov/item/2017218620/
3677,"['Chicago, Ill. : Polish National Alliance, 19...",1902,1902-1913,"['Began with rok 21, no. 1 (Stycznia 2, 1902);...","<p>\n\tIn 1881, <em>Zgoda</em> (&#8220;Harmony...",['University of Illinois at Urbana-Champaign L...,"['polish', 'english']","[41.88425, -87.63245]","['wisconsin', 'chicago', 'illinois', 'united s...",2017218621,"['Polish people--United States--Newspapers', '...",Zgoda : Wydanie dla mężczyzn,https://www.loc.gov/item/2017218621/
3678,"['Chicago, Illinois : Polish National Alliance...",1914,1914-current,"['Quarterly, January/March 2014- Began with ro...","<p>\n\tIn 1881, <em>Zgoda</em> (&#8220;Harmony...",['University of Illinois at Urbana-Champaign L...,"['polish', 'english']","[41.88425, -87.63245]","['wisconsin', 'chicago', 'illinois

In [36]:
df = raw
df

,created_published,date,dates_of_publication,description,essay,essay_contributor,language,latlong,location,raw_lccn,subjects,title,url
0,"['Abbeville, S.C. : Charles H. Allen, 1847-186...",1847,1847-1869,"['Weekly Vol. 4, no. 1 (Mar. 3, 1847)-v. 25, n...","<p>\n\tFor nearly a century, the <em>Abbeville...","['University of South Carolina; Columbia, SC']",['english'],"[34.17895, -82.38025]","['united states', 'south carolina', 'abbeville...",sn 85026945,"['Abbeville (S.C.)--Newspapers', 'Abbeville Co...",The Abbeville banner.,https://www.loc.gov/item/sn85026945/
1,"['Abbeville, S.C. : Hugh Wilson']",1865,1865-1865,['Weekly Began in July 1865. Ceased with Aug. ...,<p>\n\tThe short-lived weekly <em>Abbeville Bu...,"['University of South Carolina; Columbia, SC']",['english'],"[34.17895, -82.38025]","['united states', 'abbeville county', 'abbevil...",sn 93067670,"['Abbeville County (S.C.)--Newspapers', 'South...",The Abbeville bulletin.,https://www.loc.gov/item/sn93067670/
2,"['Abbeville, S.C. : Bonham and Perrin']",1884,1884-1887,"['Weekly Began Oct. 1, 1884; ceased in 1887. C...",<p>\n\tThe <em>Abbeville Messenger</em> (1884-...,"['University of South Carolina; Columbia, SC']",['english'],"[34.17895, -82.38025]","['united states', 'abbeville county', 'abbevil...",sn 93067668,"['Abbeville County (S.C.)--Newspapers', 'South...",The Abbeville messenger.,https://www.loc.gov/item/sn93067668/
3,"['Abbeville, S.C. : W.A. Lee & Hugh Wilson, 18...",1869,1869-1924,"['Triweekly, Jan. 7, 1920-Feb. 13, 1924 Began ...","<p>\n\tFor nearly a century, the <em>Abbeville...","['University of South Carolina; Columbia, SC']",['english'],"[34.17895, -82.38025]","['united states', 'south carolina', 'abbeville...",sn 84026853,"['Abbeville (S.C.)--Newspapers', 'Abbeville Co...",The Abbeville press and banner.,https://www.loc.gov/item/sn84026853/
4,"['Abbeville, S.C. : W.A. Lee and Hugh Wilson, ...",1860,1860-1869,"['Weekly Vol. 8, no. 28 (Nov. 9, 1860)-v. 17, ...","<p>\n\tFor nearly a century, the <em>Abbeville...","['University of South Carolina; Columbia, SC']",['english'],"[34.17895, -82.38025]","['united states', 'south carolina', 'abbeville...",sn 85042527,"['Abbeville (S.C.)--Newspapers', 'Abbeville Co...",Abbeville press.,https://www.loc.gov/item/sn85042527/
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3675,"['Allegheny, Pa. : Narod. Hrv. Zajednica']",1894,1894-current,['Weekly (except the second week in July and t...,<p>\n\tThe&#160;<em>Zajedni</em><em>&#269;</em...,"['Ohio Historical Society, Columbus, OH']","['english', 'croatian']","[41.0981, -80.65079]","['mahoning', 'mahoning county', 'pennsylvania'...",sn 84024547,"['Croats--United States--Newspapers', 'Croatia...",Zajedničar = Fraternalist.,https://www.loc.gov/item/sn84024547/
3676,"['Chicago, Ill. : Polish National Alliance, 19...",1900,1900-1913,"['Began with rok 19, no. 1 (13 Stycznia, 1900)...","<p>\n\tIn 1881, <em>Zgoda</em> (&#8220;Harmony...",['University of Illinois at Urbana-Champaign L...,"['polish', 'english']","[41.88425, -87.63245]","['wisconsin', 'chicago', 'illinois', 'united s...",2017218620,"['Polish people--United States--Newspapers', '...",Zgoda : Wydania dla niewiast,https://www.loc.gov/item/2017218620/
3677,"['Chicago, Ill. : Polish National Alliance, 19...",1902,1902-1913,"['Began with rok 21, no. 1 (Stycznia 2, 1902);...","<p>\n\tIn 1881, <em>Zgoda</em> (&#8220;Harmony...",['University of Illinois at Urbana-Champaign L...,"['polish', 'english']","[41.88425, -87.63245]","['wisconsin', 'chicago', 'illinois', 'united s...",2017218621,"['Polish people--United States--Newspapers', '...",Zgoda : Wydanie dla mężczyzn,https://www.loc.gov/item/2017218621/
3678,"['Chicago, Illinois : Polish National Alliance...",1914,1914-current,"['Quarterly, January/March 2014- Began with ro...","<p>\n\tIn 1881, <em>Zgoda</em> (&#8220;Harmony...",['University of Illinois at Urbana-Champaign L...,"['polish', 'english']","[41.88425, -87.63245]","['wisconsin', 'chicago', 'illinois

In [37]:
# remove html tags

def stripped_strings(text):
    try:
        return (' '.join(BeautifulSoup(text).stripped_strings))
    except TypeError as e:
        print(e)


df['essay'] = [stripped_strings(text) for text in df['essay'] ]
    

object of type 'float' has no len()


In [38]:
df

,created_published,date,dates_of_publication,description,essay,essay_contributor,language,latlong,location,raw_lccn,subjects,title,url
0,"['Abbeville, S.C. : Charles H. Allen, 1847-186...",1847,1847-1869,"['Weekly Vol. 4, no. 1 (Mar. 3, 1847)-v. 25, n...","For nearly a century, the Abbeville Press and ...","['University of South Carolina; Columbia, SC']",['english'],"[34.17895, -82.38025]","['united states', 'south carolina', 'abbeville...",sn 85026945,"['Abbeville (S.C.)--Newspapers', 'Abbeville Co...",The Abbeville banner.,https://www.loc.gov/item/sn85026945/
1,"['Abbeville, S.C. : Hugh Wilson']",1865,1865-1865,['Weekly Began in July 1865. Ceased with Aug. ...,The short-lived weekly Abbeville Bulletin (Jul...,"['University of South Carolina; Columbia, SC']",['english'],"[34.17895, -82.38025]","['united states', 'abbeville county', 'abbevil...",sn 93067670,"['Abbeville County (S.C.)--Newspapers', 'South...",The Abbeville bulletin.,https://www.loc.gov/item/sn93067670/
2,"['Abbeville, S.C. : Bonham and Perrin']",1884,1884-1887,"['Weekly Began Oct. 1, 1884; ceased in 1887. C...",The Abbeville Messenger (1884-87) was one of s...,"['University of South Carolina; Columbia, SC']",['english'],"[34.17895, -82.38025]","['united states', 'abbeville county', 'abbevil...",sn 93067668,"['Abbeville County (S.C.)--Newspapers', 'South...",The Abbeville messenger.,https://www.loc.gov/item/sn93067668/
3,"['Abbeville, S.C. : W.A. Lee & Hugh Wilson, 18...",1869,1869-1924,"['Triweekly, Jan. 7, 1920-Feb. 13, 1924 Began ...","For nearly a century, the Abbeville Press and ...","['University of South Carolina; Columbia, SC']",['english'],"[34.17895, -82.38025]","['united states', 'south carolina', 'abbeville...",sn 84026853,"['Abbeville (S.C.)--Newspapers', 'Abbeville Co...",The Abbeville press and banner.,https://www.loc.gov/item/sn84026853/
4,"['Abbeville, S.C. : W.A. Lee and Hugh Wilson, ...",1860,1860-1869,"['Weekly Vol. 8, no. 28 (Nov. 9, 1860)-v. 17, ...","For nearly a century, the Abbeville Press and ...","['University of South Carolina; Columbia, SC']",['english'],"[34.17895, -82.38025]","['united states', 'south carolina', 'abbeville...",sn 85042527,"['Abbeville (S.C.)--Newspapers', 'Abbeville Co...",Abbeville press.,https://www.loc.gov/item/sn85042527/
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3675,"['Allegheny, Pa. : Narod. Hrv. Zajednica']",1894,1894-current,['Weekly (except the second week in July and t...,The Zajedni č ar (“Fraternalist”) was establis...,"['Ohio Historical Society, Columbus, OH']","['english', 'croatian']","[41.0981, -80.65079]","['mahoning', 'mahoning county', 'pennsylvania'...",sn 84024547,"['Croats--United States--Newspapers', 'Croatia...",Zajedničar = Fraternalist.,https://www.loc.gov/item/sn84024547/
3676,"['Chicago, Ill. : Polish National Alliance, 19...",1900,1900-1913,"['Began with rok 19, no. 1 (13 Stycznia, 1900)...","In 1881, Zgoda (“Harmony”) appeared as a weekl...",['University of Illinois at Urbana-Champaign L...,"['polish', 'english']","[41.88425, -87.63245]","['wisconsin', 'chicago', 'illinois', 'united s...",2017218620,"['Polish people--United States--Newspapers', '...",Zgoda : Wydania dla niewiast,https://www.loc.gov/item/2017218620/
3677,"['Chicago, Ill. : Polish National Alliance, 19...",1902,1902-1913,"['Began with rok 21, no. 1 (Stycznia 2, 1902);...","In 1881, Zgoda (“Harmony”) appeared as a weekl...",['University of Illinois at Urbana-Champaign L...,"['polish', 'english']","[41.88425, -87.63245]","['wisconsin', 'chicago', 'illinois', 'united s...",2017218621,"['Polish people--United States--Newspapers', '...",Zgoda : Wydanie dla mężczyzn,https://www.loc.gov/item/2017218621/
3678,"['Chicago, Illinois : Polish National Alliance...",1914,1914-current,"['Quarterly, January/March 2014- Began with ro...","In 1881, Zgoda (“Harmony”) appeared as a weekl...",['University of Illinois at Urbana-Champaign L...,"['polish', 'english']","[41.88425, -87.63245]","['wisconsin', 'chicago', 'illinois

In [234]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/danielevans/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [44]:
# convert to string, not sure why a certain number of essays are being fed in as a float.
df["essay"] = df["essay"].astype(str)

In [45]:
# not necessary, debug later
start = time.time()
df["unigrams"] = df["essay"].apply(nltk.word_tokenize)
print(time.time() - start)

4.225123167037964


In [46]:
df

,created_published,date,dates_of_publication,description,essay,essay_contributor,language,latlong,location,raw_lccn,subjects,title,url,unigrams
0,"['Abbeville, S.C. : Charles H. Allen, 1847-186...",1847,1847-1869,"['Weekly Vol. 4, no. 1 (Mar. 3, 1847)-v. 25, n...","For nearly a century, the Abbeville Press and ...","['University of South Carolina; Columbia, SC']",['english'],"[34.17895, -82.38025]","['united states', 'south carolina', 'abbeville...",sn 85026945,"['Abbeville (S.C.)--Newspapers', 'Abbeville Co...",The Abbeville banner.,https://www.loc.gov/item/sn85026945/,"[For, nearly, a, century, ,, the, Abbeville, P..."
1,"['Abbeville, S.C. : Hugh Wilson']",1865,1865-1865,['Weekly Began in July 1865. Ceased with Aug. ...,The short-lived weekly Abbeville Bulletin (Jul...,"['University of South Carolina; Columbia, SC']",['english'],"[34.17895, -82.38025]","['united states', 'abbeville county', 'abbevil...",sn 93067670,"['Abbeville County (S.C.)--Newspapers', 'South...",The Abbeville bulletin.,https://www.loc.gov/item/sn93067670/,"[The, short-lived, weekly, Abbeville, Bulletin..."
2,"['Abbeville, S.C. : Bonham and Perrin']",1884,1884-1887,"['Weekly Began Oct. 1, 1884; ceased in 1887. C...",The Abbeville Messenger (1884-87) was one of s...,"['University of South Carolina; Columbia, SC']",['english'],"[34.17895, -82.38025]","['united states', 'abbeville county', 'abbevil...",sn 93067668,"['Abbeville County (S.C.)--Newspapers', 'South...",The Abbeville messenger.,https://www.loc.gov/item/sn93067668/,"[The, Abbeville, Messenger, (, 1884-87, ), was..."
3,"['Abbeville, S.C. : W.A. Lee & Hugh Wilson, 18...",1869,1869-1924,"['Triweekly, Jan. 7, 1920-Feb. 13, 1924 Began ...","For nearly a century, the Abbeville Press and ...","['University of South Carolina; Columbia, SC']",['english'],"[34.17895, -82.38025]","['united states', 'south carolina', 'abbeville...",sn 84026853,"['Abbeville (S.C.)--Newspapers', 'Abbeville Co...",The Abbeville press and banner.,https://www.loc.gov/item/sn84026853/,"[For, nearly, a, century, ,, the, Abbeville, P..."
4,"['Abbeville, S.C. : W.A. Lee and Hugh Wilson, ...",1860,1860-1869,"['Weekly Vol. 8, no. 28 (Nov. 9, 1860)-v. 17, ...","For nearly a century, the Abbeville Press and ...","['University of South Carolina; Columbia, SC']",['english'],"[34.17895, -82.38025]","['united states', 'south carolina', 'abbeville...",sn 85042527,"['Abbeville (S.C.)--Newspapers', 'Abbeville Co...",Abbeville press.,https://www.loc.gov/item/sn85042527/,"[For, nearly, a, century, ,, the, Abbeville, P..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3675,"['Allegheny, Pa. : Narod. Hrv. Zajednica']",1894,1894-current,['Weekly (except the second week in July and t...,The Zajedni č ar (“Fraternalist”) was establis...,"['Ohio Historical Society, Columbus, OH']","['english', 'croatian']","[41.0981, -80.65079]","['mahoning', 'mahoning county', 'pennsylvania'...",sn 84024547,"['Croats--United States--Newspapers', 'Croatia...",Zajedničar = Fraternalist.,https://www.loc.gov/item/sn84024547/,"[The, Zajedni, č, ar, (, “, Fraternalist, ”, )..."
3676,"['Chicago, Ill. : Polish National Alliance, 19...",1900,1900-1913,"['Began with rok 19, no. 1 (13 Stycznia, 1900)...","In 1881, Zgoda (“Harmony”) appeared as a weekl...",['University of Illinois at Urbana-Champaign L...,"['polish', 'english']","[41.88425, -87.63245]","['wisconsin', 'chicago', 'illinois', 'united s...",2017218620,"['Polish people--United States--Newspapers', '...",Zgoda : Wydania dla niewiast,https://www.loc.gov/item/2017218620/,"[In, 1881, ,, Zgoda, (, “, Harmony, ”, ), appe..."
3677,"['Chicago, Ill. : Polish National Alliance, 19...",1902,1902-1913,"['Began with rok 21, no. 1 (Stycznia 2, 1902);...","In 1881, Zgoda (“Harmony”) appeared as a weekl...",['University of Illinois at Urbana-Champaign L...,"['polish', 'english']","[41.88425, -87.63245]","['wisconsin', 'chicago', 'illinois', 'united s...",2017218621,"['Polish people--United States--Newspapers', '...",Zgo

In [47]:
from nltk.text import Text

In [48]:
text = df['unigrams'].apply(Text)

In [49]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/danielevans/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [50]:
token_sent = df.essay.apply(nltk.word_tokenize)  
tagged_sent= token_sent.apply(nltk.pos_tag)

In [51]:
df['essay']

0       For nearly a century, the Abbeville Press and ...
1       The short-lived weekly Abbeville Bulletin (Jul...
2       The Abbeville Messenger (1884-87) was one of s...
3       For nearly a century, the Abbeville Press and ...
4       For nearly a century, the Abbeville Press and ...
                              ...                        
3675    The Zajedni č ar (“Fraternalist”) was establis...
3676    In 1881, Zgoda (“Harmony”) appeared as a weekl...
3677    In 1881, Zgoda (“Harmony”) appeared as a weekl...
3678    In 1881, Zgoda (“Harmony”) appeared as a weekl...
3679    In 1881, Zgoda (“Harmony”) appeared as a weekl...
Name: essay, Length: 3680, dtype: object

In [52]:
token_sent

0       [For, nearly, a, century, ,, the, Abbeville, P...
1       [The, short-lived, weekly, Abbeville, Bulletin...
2       [The, Abbeville, Messenger, (, 1884-87, ), was...
3       [For, nearly, a, century, ,, the, Abbeville, P...
4       [For, nearly, a, century, ,, the, Abbeville, P...
                              ...                        
3675    [The, Zajedni, č, ar, (, “, Fraternalist, ”, )...
3676    [In, 1881, ,, Zgoda, (, “, Harmony, ”, ), appe...
3677    [In, 1881, ,, Zgoda, (, “, Harmony, ”, ), appe...
3678    [In, 1881, ,, Zgoda, (, “, Harmony, ”, ), appe...
3679    [In, 1881, ,, Zgoda, (, “, Harmony, ”, ), appe...
Name: essay, Length: 3680, dtype: object

In [53]:
text = Text(txt)

NameError: name 'txt' is not defined

In [54]:
# count of words 
t = token_sent.apply(lambda x: nltk.Text(x).count('century'))
t

0       2
1       0
2       1
3       2
4       2
       ..
3675    0
3676    0
3677    0
3678    0
3679    0
Name: essay, Length: 3680, dtype: int64

In [84]:
# concordance search
t = token_sent.apply(lambda x: nltk.Text(x).concordance("african"))

Displaying 1 of 1 matches:
 included : the establishment of an African American school , called Ferguson A
no matches
no matches
Displaying 1 of 1 matches:
 included : the establishment of an African American school , called Ferguson A
Displaying 1 of 1 matches:
 included : the establishment of an African American school , called Ferguson A
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
Displaying 1 of 1 matches:
eodore Roosevelt ’ s appointment of African Americans to federal positions in t
no matches
no matches
no matches
no matches
Displaying 2 of 2 matches:
te examples of inequalities between African Americans and whites . For instance
s higher for whites than it was for African American children . The Advance als
Displaying 2 of 2 matches:
tor Hiram Rhodes Revels , the first African American to serve in the U.S. Congr
ection Law of 1902 , which excluded African Americans from voting in a primary 


no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
Displaying 1 of 1 matches:
 Duluth working to quash distinctly African American prostitution , Morrison ’ 
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
Displaying 2 of 2 matches:
eel collection containing issues of African American newspapers published in th
ew a complete list of all digitized African American titles available in the Ch
Displaying 4 of 4 matches:
xt twenty-years saw a diminution of African American journals as rigid racial s
 and affordable burial insurance to African Ameri

ewman , a leader in the Twin Cities African American community , who used his p
 , claimed to be the first and only African American magazine in the northweste
ern United States . The Twin Cities African American community was relatively s
l scandal and gossip among both the African American and white communities . Wh
ibers , and circulation grew as the African American population in the Twin Cit
1950 , Minnesota was home to 14,000 African Americans , and the newspapers clai
industries and unions that excluded African American workers , investigated and
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
Displaying 4 of 4 matches:
issionary Record ( 1868-79 ? ) , an African American-owned Republican newspaper
 , the son of a prominent family of African and Sephardic Jewish ancestry . In 
the First Episcopal District of the African Methodist Episcopal Church , which 
he South Carolina Conference of the Afric

no matches
no matches
no matches
no matches
no matches
Displaying 1 of 1 matches:
h the U.S. Supreme Court ruled that African Americans were not citizens and lac
no matches
no matches
no matches
no matches
no matches
no matches
Displaying 2 of 2 matches:
eel collection containing issues of African American newspapers published in th
ew a complete list of all digitized African American titles available in the Ch
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
Displaying 5 of 5 matches:
The Star was a staunchly Democratic African American paper in an era when Repub
a Daily Star , the paper championed African American causes , promoting progres
s efforts to improve conditions for African Americans . At the time of the race
ace massacre , approximately 11,000 African Americans lived in Tulsa . They own
 criticized its actions towards the African American community . This distrust 
no matches
no matches
no matches
no matches
no matches
no m

In [307]:
####

In [63]:
tagged_sent

0       [(For, IN), (nearly, RB), (a, DT), (century, N...
1       [(The, DT), (short-lived, JJ), (weekly, JJ), (...
2       [(The, DT), (Abbeville, NNP), (Messenger, NNP)...
3       [(For, IN), (nearly, RB), (a, DT), (century, N...
4       [(For, IN), (nearly, RB), (a, DT), (century, N...
                              ...                        
3675    [(The, DT), (Zajedni, NNP), (č, NNP), (ar, NN)...
3676    [(In, IN), (1881, CD), (,, ,), (Zgoda, NNP), (...
3677    [(In, IN), (1881, CD), (,, ,), (Zgoda, NNP), (...
3678    [(In, IN), (1881, CD), (,, ,), (Zgoda, NNP), (...
3679    [(In, IN), (1881, CD), (,, ,), (Zgoda, NNP), (...
Name: essay, Length: 3680, dtype: object

In [64]:
nlp = spacy.load("en_core_web_sm")

NameError: name 'spacy' is not defined

In [65]:
!pip install spacy
import spacy

In [66]:
!python3 -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 10.4 MB/s eta 0:00:00m eta 0:00:010:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [67]:
nlp = spacy.load("en_core_web_sm")

In [82]:
## Need to write this so each line goes to the df

df['essay']= df['essay'].astype(str)


for row in range(len(df)):
    doc = nlp(df.loc[row, "essay"])    
#     for np in doc.noun_chunks:
#         print(np.text)
#     for ent in doc.ents:
#         if ent.label_ == 'PERSON':
#             print(ent.text)
    people = [ent.text for ent in doc.ents if ent.label_ == 'PERSON']
    orgs = [ent.text for ent in doc.ents if ent.label_ == 'ORG']
    LOC = [ent.text for ent in doc.ents if ent.label_ == 'LOC']


Abbeville
Calhoun Falls
Greenwood
Banner
Charles Henry Allen
Henry S. Kerr
Coleman C. Puckett
George W. Fant
Puckett
Hugh Wilson
Jr.
William Augustus Lee
Lee
George W. Fant
Hugh Wilson
Banner
William Augustus Lee
Fred Devereaux West
Hugh Wilson
Abbeville Banner
Greenville Patriot
James L. Orr
Hugh Wilson
William Augustus Lee
Lee
Abbeville Messenger
Banner
Abbeville Messenger
Saluda Argus
Saluda Argus
Milledge Lipscomb Bonham
James Sumter
Thomas Perrin Cothran
Saluda Argus
Abbeville Messenger
Abbeville Messenger
Anderson Intelligencer
Bonham
Watchman
Abbeville Messenger
Abbeville Messenger
Bonham
Abbeville Messenger
Abbeville
Calhoun Falls
Greenwood
Banner
Charles Henry Allen
Henry S. Kerr
Coleman C. Puckett
George W. Fant
Puckett
Hugh Wilson
Jr.
William Augustus Lee
Lee
George W. Fant
Hugh Wilson
Banner
William Augustus Lee
Fred Devereaux West
Abbeville
Calhoun Falls
Greenwood
Banner
Charles Henry Allen
Henry S. Kerr
Coleman C. Puckett
George W. Fant
Puckett
Hugh Wilson
Jr.
William Aug

James Madison
Alexandria
Lawrence Washington
Lawrence
George Washington
Alexandria
Alexandria
Alexandria
John Corse
Nathaniel Rounsavell
Henry Pittman
Corse
Alexandria
Solidaritet
Carl Ahlteen
Carl Skoglund
Ragnar Johanson
Edward Mattson
Signe Aurell
Johanson
Solidaritet
Josef Hillström
Joe Hill
Carl Ahlteen
Sigfrid Stenberg
Ragnar Johanson
Edward Mattson
Johanson
Johanson
Facklan
Engdahl
Carl Skogland
Floyd B. Olson
colonists
Rees Lloyd
Andrew Carnegie
George W.N. Bolsinger
James Todd Hutchinson
Hutchinson
Gazette
Abraham Andrews Barker
Hutchinson
Barker
Barker
James Todd Hutchinson
William E. Hutchinson
William
James Hutchinson
Edmund James
George P. Rowell & Company’s
James A. Edgerton
Powhatan Ellis
Ellisville
Newton Knight
Ellisville
Washington L. Lee
James J. Shannon
Edgar G. Harris
Greenwood
Harris
Ellisville
Ovett
Ovett
Ellisville
Thomas J. O’Keefe
Theodore Roosevelt
George P. Rowell & Company’s
James A. Edgerton
M.L. Hart
Miss C. M. Fleming
Charles F. Brown
Brown
Brown
Brown
C

Arizona Miner
Richard McCormick
Tisdale A. Hand
McCormick
William H. Lyon
McCormick
Miner
John H. Marion
Miner
Miner
Benjamin H. Weaver
Miner
Thomas J. Butler
Charles W. Beach
Samuel N. Holmes
Holmes
Sherman House
John C. Martin
Martin
John W. Milnes
Arthur John Doud
A. V. Napier
Myer
Rebecca Staman Rutz
Bat Mitzvahs
Rosh Hashanah
Myer
Rebecca Rutz
Joseph Gumbiner
Marcus Breger
Myer Rutz
Abe
Chanin
Abe
Charles.
O. Ziegenfuss
Edwin S. Gill
Lewis Wolfley
Gill
Prescott
Gill
Frank M. Murphy
Charles C. Randolph
George W. Vickers
Stephen Weaver Higley
Sims Ely
John C. Adams
Dwight Heard
Charles A. Stauffer
Theodore Roosevelt
Eugene Pulliam
Gazette Pulliam
David A. Gordon
C. L. Minor
Minor
William H. Lyon
John W. Dorrington
Sentinel
Sentinel
William Harold Shorey
Roy Dennis
Karl E. Allen
David A. Gordon
C. L. Minor
Minor
William H. Lyon
John W. Dorrington
Sentinel
Sentinel
William Harold Shorey
Roy Dennis
Karl E. Allen
David A. Gordon
C. L. Minor
Minor
William H. Lyon
John W. Dorrington
Senti

Bamberg Herald
Bamberg Cotton Mill
Henry Simms Hartzog
Benjamin Ryan Tillman
I.R. Hungry
David Graham Copeland
Henry Hartzog
Arthur Buist
Jordan
Henry Calhoun
Bamberg Cotton Mills
Aquilla Knight
Herald
Carl Kilgus
Betty
Carl Kilgus
James Nelson Turner
David Llewellyn Morgan
John Bell
James Turner’s
George C. Benham
Morgan
Samuel Blakely Kennedy
Kennedy
Ezekiel Ezra Smith
Grover Cleveland
George Allen Mebane
John H. Williamson
Williamson
Williamson
Abbeville
Calhoun Falls
Greenwood
Banner
Charles Henry Allen
Henry S. Kerr
Coleman C. Puckett
George W. Fant
Puckett
Hugh Wilson
Jr.
William Augustus Lee
Lee
George W. Fant
Hugh Wilson
Banner
William Augustus Lee
Fred Devereaux West
Baraboo
Levi Cook
John Kartack
A. D. Dorsett
Harry Ellsworth Cole
Dorsett
Baraboo
Levi Cook
John Kartack
A. D. Dorsett
Harry Ellsworth Cole
Dorsett
Michael L. Sherpy
Thomas Barber
Barber
William Cutler
Edward “Wiley” Payne
Edgar P. Caruthers
William G. Musgrove
Henry C. Walker
Charles C. Painter
Herr
Medicine Lodg

A. Lounsberry
George Armstrong Custer
Mark Kellogg
Custer
Clement Lounsberry
John Carnahan
Jewel
A. Lounsberry
George Armstrong Custer
Mark Kellogg
Custer
Clement Lounsberry
John Carnahan
Jewel
A. Lounsberry
George Armstrong Custer
Mark Kellogg
Custer
Clement Lounsberry
John Carnahan
Jewel
A. Lounsberry
George Armstrong Custer
Mark Kellogg
Custer
Clement Lounsberry
John Carnahan
Jewel
F. E. Eardman
A. Eardman
G. W. Barrows
Edward B. Reed
Edward Reed
John Z. Reed
Shelby D. Reed
Arthur W. Gird
John Reed
George L. McManus
J. C. Crandall
J. C. Walker
Edmund Shaw
F.E. Ross
R. B. French
Wallace H. Burden
Louis C. Baber
G. W. Stephenson
C. A. Stephenson
George F. Robb
F. E. Eardman
A. Eardman
G. W. Barrows
Edward B. Reed
Edward Reed
John Z. Reed
Shelby D. Reed
Arthur W. Gird
John Reed
George L. McManus
J. C. Crandall
J. C. Walker
Edmund Shaw
F.E. Ross
R. B. French
Wallace H. Burden
Louis C. Baber
G. W. Stephenson
C. A. Stephenson
George F. Robb
M.L. Robinson
V.B. Beckett
The Black Range
Tranq

...

Newark Advertiser
Newark Advertiser
James Martin
Newark Advertiser
Newark Advertiser
Martin
Newark Advertiser
William
R.B. Smith
Nemiah Northrop
Raymond Rees
R.B. Smith
S.G. Rees
William H. Newell
Andrew Johnson
Frank J. Parker
E.E. Fall
Percy Holland
Arthur Green
Levi Ankeny
Adele Evans
Adele Evans Returns
Charles Edward Warburton
James Barclay Harding
Harding
Warburton
Harding
Warburton
Watson Armbruster
Warburton
Andrew Johnson
Warburton
Barclay Harding Warburton
Charles Austen Bates
Warburton
Warburton
Bates
Warburton
Rodman Wanamaker
John Wanamaker
John T. Windrim
Warburton
George A. Waite
Henry Anson
Marshall
Marshall
Edwin N. Chapin
Royal H. Barnhart
Arthur S. Burnell
Thomas Mercer
Thomas E. McCracken
Marshall
Halsey Lee Merriman
Burnell
Silas C. McFarland
David Windsor Norris Jr.
Theodore Johnston
William P. Hughes
George F. Thayer
Rodney C. Wells
Frank A. Moscrip
Harry J. Rodgers
Les Grandes Fourches
George B. Winship
Alexander McKenzie
McKenzie
Jerimiah Dempster Bacon
Norman 

William E. Bradley
John W. Woltz
Harry Freese
John Freese
Harry Freese
John Freese
Albert L. Clark
Timothy Hurley
George Presbury Rowell
James Henry Harris
Daniel Henry Chamberlain
Robert Woody
Timothy Hurley
William King
A. G. Russell
George P. Rowell
Russell
John Rhodes Oliphant
G. S. Harmon
River Watchman
Russell
Lucius Olen Crosby
Theodore Gilmore Bilbo
Bilbo
James Melyn Latta
Hiram Reed
Lyman Stickney
James Gordon Thompson
James Latta
Joseph Wilkes
James Thompson
William Henry Seward
John E. Hayes
George F. Weaver
John Hise
William Osman
Osman
Osman
William Osman
John Hise
Osman
William Osman
Osman
Douglas Hapeman
Osman
Alonzo Delano
William
Osman
Delano
Delano
Mark Twain
Bret Harte
Delano
Alonzo Delano’s
George F. Weaver
John Hise
William Osman
Osman
Osman
William Osman
John Hise
Osman
William Osman
Osman
Douglas Hapeman
Osman
Alonzo Delano
William
Osman
Delano
Delano
Mark Twain
Bret Harte
Delano
Alonzo Delano’s
Henry C. Bradsby
Thomas A. Buckley
D. S. Buckley
Buckley
Upper Lehig

William T. Gibbs
James D. Carpenter
Carpenter
Taylor
Carpenter
Belmont Perry
Perry
Carpenter
Carpenter
Carpenter
Woodbury
Woodrow Wilson
Edmund H. Carpenter
Edmund Carpenter
Cornish
John H. Mundall
Hiram R. Hawkins
Hawkins
Myron Angel
John P. Jones
William Sharon
James Nye
Jones
Jones
Sharon
Sharon
Joe Goodman
Sharon
Sharon
Adolph Sutro
Charles C. Stevenson
Thaddeus R. Manning
Wilmington Messenger
Samuel Thomas Peace
Taylor Way
Rio Bonito
Saturnino Baca
Lawrence G. Murphy
William Brady
J.H. Blazer
Lincoln
Jones Taliaferro
Jones Taliaferro
M.S. Taliaferro
Grover Cleveland
a W. Cal.
Willis Elbert Mollison
Mollison
Henry L. Slaughter
Mollison
Mollison
Chronicle
W. A. Young
Brinton
Job W.
Charles M.
Young
Brintons
Chronicle
J.W. Brinton’s
Brinton
Mergenthaler
J.W. Brinton’s
Brinton
Brinton
Clarence T. Bolstad
Goldfield
Goldfield
James F. O’Brien
Tonopah Miner
J.P. Loftus
Charles Sprague
William W. Booth
George Wingfield
John C. Martin
Joseph Edward Robinson
Luther Melancthan Nash
Goldsboro

Thomas Cooper
Rollin Cooper
Cooper
Ed Stair
Sheyenne
Fredrick H. Adams
Percy R. Trubshaw
H. S. Rearick
Thomas Cooper
Rollin Cooper
Cooper
Ed Stair
Sheyenne
Fredrick H. Adams
Percy R. Trubshaw
H. S. Rearick
Arthur Harper Grimshaw
John Stradley
Grimshaw
Grimshaw
Grimshaw
Holly Springs
Hiram G. Runnels
Andrew Jackson
Spring
Jefferson Davis
William Loring
Guerilla
Guerilla
Guerilla
Loring
Guerilla
Rebel
Guerilla
Rebel
Guerilla
Guerilla
Abraham Lincoln
Guerilla
Guerilla
Guerilla
R. Don McLeod
Herbert S. Moore
C.K. Allen
Anna Karenina
Roy Hoffman
Guthrie
Hoffman
Benjamin Harrison
Guthrie
Guthrie
Hoffman
Frank Greer’s
Leslie Niblack
Niblack
Louisville Courier-Journal
Guthrie
Guthrie
Niblack
Lewis Haines Wentz
Raymond Fields
Phil McMullen
The Hammond Gazette
William A. Hammond
The Hammond Gazette
Gazette
Charley Greer
Gazette
George Everett
Captain Everett
Gazette
Everett
Gazette
John Jacob Omenhausser
Samuel A. Spear
David Ross Locke
Findlay
Locke
Locke
Nasby
Locke
Abraham Lincoln
Findlay
Eli

KeyboardInterrupt: 

In [76]:
people

['Dziennik',
 'Dziennik Chicagoski',
 'Dziennik Związkowy',
 'Gazeta Polska Katolicka',
 'Zasady w Polityce',
 'Zapowiesci',
 'Jezy']

In [80]:
ppl

[]

In [365]:
org

['the Abbeville Press',
 'Banner',
 'The N.W. Ayer & Son’s',
 'the Seaboard Air Line',
 'the Abbeville Cotton Mill Company',
 'Ferguson Academy',
 'Harbison Agricultural College',
 'the Seaboard Air Line',
 'The Abbeville Press',
 'Abbeville Press',
 'the Abbeville Banner',
 'the Abbeville Independent Press',
 'the Abbeville Press',
 'the Abbeville Independent Press',
 'the Anderson Southern Rights Advocate',
 'Fant',
 'the Anderson Gazette',
 'Abbeville Banner',
 'the Abbeville Press',
 'the Confederate States Army',
 'Wilson',
 'the Abbeville Banner',
 'the Abbeville Press',
 'the Abbeville Press',
 'Banner',
 'the Press and Banner',
 'Press',
 'Banner',
 'the Independent Press',
 'the National Bank of Anderson',
 'Jr.',
 'Press',
 'the South Carolina Press Association',
 'the Abbeville Press',
 'Banner',
 'Press',
 'Press']

# todo:
0. write this right.
1. check for and delete csv if does exist
2. (done) refactor / rate limit api calls.
3. (done) write to list or pickle before writing to csv
4. (done) test which sleep function is necessary
5. document code
6. NER / BERT
7. Begin planning for fall / next steps
8. Ingest / Clean xlsx
9. binderhub
10. send to M 9/2